## filter the .obj file to only keep the objects within the bounding box


In [1]:
import trimesh as tm
import numpy as np
import pandas as pd
import pprint

In [14]:
context_path = os.path.relpath('../data/OBJ_files/building_envelope.obj')

envelope_mesh = tm.load_mesh(context_path, process=False)

all_vertices = envelope_mesh.vertices.tolist()
vertices_df = pd.DataFrame(all_vertices)

#return true for all vertices that fall within the bounding box: 
b_box =[83738.31556157247, 445723.7648830635, 84831.3496488015, 447090.82246806903] #made smaller for trial
# b_box = [83738.31556157247, 445723.7648830635, 84831.3496488015, 448090.82246806903] # full b_box


vertices_df['xmin'] = b_box[0] < vertices_df[0] 
vertices_df['xmax'] = vertices_df[0] < b_box[2]
vertices_df['ymin'] = b_box[1] < vertices_df[1]
vertices_df['ymax'] = vertices_df[1] < b_box[3]


In [15]:
# add indices-column
vertices_df.reset_index(inplace=True)

# remove all vertices that do not fall within the bounding box
condition_matrix = vertices_df[['xmin', 'xmax', 'ymin', 'ymax']].to_numpy()
vertices_df['to_keep'] = (condition_matrix.sum(axis=1) == 4)
bboxed_vertices = vertices_df[(vertices_df.to_keep == True )]

In [16]:
bboxed_vertices = bboxed_vertices.drop(['xmin', 'xmax', 'ymin', 'ymax', 'to_keep'], axis = 1)
bboxed_vertices = bboxed_vertices.to_numpy()

In [18]:

lst = [0]*len(bboxed_vertices)
for i, elem in enumerate(bboxed_vertices):
    lst[i] = int(elem[0])

translationlist = [0]*len(bboxed_vertices)
for i, elem in enumerate(bboxed_vertices):
    translationlist[i] = [i, int(elem[0])]

final_bounded_vertices = [0]*len(bboxed_vertices)
for i, elem in enumerate(bboxed_vertices):
    final_bounded_vertices[i] = [elem[1], elem[2], elem[3]]

In [19]:
y = [elem for elem in lst]
faces = envelope_mesh.faces.tolist()

bounded_faces = []

for x in faces:
    if len(set(x+y))==len(y):
        bounded_faces.append(x)

In [20]:
flatList = [item for elem in bounded_faces for item in elem]
newlist = []
for val in flatList: 
    if val in lst: 
        newval = lst.index(val)
        newlist.append(newval)

final_bounded_faces = [newlist[i:i + 3] for i in range(0, len(newlist), 3)]
final_bounded_faces = np.array(final_bounded_faces)

In [21]:
combined_mesh = tm.Trimesh(final_bounded_vertices, final_bounded_faces)

In [22]:
obj = tm.exchange.obj.export_obj(combined_mesh)

In [23]:
file_name = '../data/OBJ_files/TEST.obj'
with open(file_name, 'w') as f:
    f.write(obj)